### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [120]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
125,125,Yathecal82,20,Female,62,Piece Maker,1.87
548,548,Yathecal82,20,Female,75,Brutality Ivory Warmace,2.42
727,727,Yathecal82,20,Female,104,Gladiator's Glaive,1.93


## Player Count

* Display the total number of players


In [20]:
total_players = len(purchase_data["SN"].unique())
pd.DataFrame([{"Total Players": total_players}])

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [135]:
#defining a function to convert value into currency format
def as_currency(amount):
        return '${:,.2f}'.format(amount)
    
total_unique_items = len(purchase_data["Item Name"].unique())
avg_price = as_currency(purchase_data["Price"].mean())
total_purchases = purchase_data["Item Name"].count()
total_revenue = as_currency(purchase_data["Price"].sum())
pd.DataFrame([{
    "Number of Unique Items" : total_unique_items,
    "Average Price" : avg_price,
    "Number of Purchases": total_purchases,
    "Total Revenue": total_revenue}])

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [84]:
# sn_gender_group = purchase_data.groupby(["SN", "Gender"])
# gender_group = sn_gender_group.count().groupby("Gender")
# gender_group.count()


male_count = purchase_data.drop_duplicates(subset = ["SN", "Gender"]).loc[purchase_data["Gender"] == "Male", "Gender"].count()
female_count = purchase_data.drop_duplicates(subset = ["SN", "Gender"]).loc[purchase_data["Gender"] == "Female", "Gender"].count()
other_count = purchase_data.drop_duplicates(subset = ["SN", "Gender"]).loc[purchase_data["Gender"] == "Other / Non-Disclosed", "Gender"].count()
total_gender = purchase_data.drop_duplicates(subset = ["SN", "Gender"]).loc[:, "Gender"].count()
male_perc = "{0:.2f}%".format((male_count/total_gender) * 100)
female_perc = "{0:.2f}%".format((female_count/total_gender) * 100)
other_perc = "{0:.2f}%".format((other_count/total_gender) * 100)

gender_dict = [{"Gender": "Male", "Total Count" : male_count, "Percentage of Players" : male_perc},
             {"Gender": "Female", "Total Count" : female_count, "Percentage of Players" : female_perc},
             {"Gender": "Other/Non-Disclosed", "Total Count" : other_count, "Percentage of Players" : other_perc}]

pd.DataFrame(gender_dict)


,Gender,Total Count,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other/Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [17]:
#use grouping on SN and Gender (removes duplicates) to get number of players of each gender
sn_gender_group = purchase_data.groupby(["SN", "Gender"])
sn_gender_n_df = sn_gender_group.count()
gender_group2 = sn_gender_n_df.groupby(["Gender"])
gender_df2 = gender_group2.count() #use for Avg total purchase per person

#group just on Gender
gender_group = purchase_data.groupby(["Gender"])
gender_df = gender_group.count()
gender_df["Purchase Count"] = gender_df[["Purchase ID"]]
gender_df["Total Purchase Value"] = gender_group.sum()["Price"]
gender_df["Average Purchase Price"] = gender_df["Total Purchase Value"]/gender_df["Purchase ID"]

#divide total purchase value by number of players of each gender
gender_df["Avg Total Purchase Per Person"] = gender_df["Total Purchase Value"]/gender_df2["Purchase ID"]

#change format (as_currency function defined earlier) and display
gender_df.iloc[:, 7:] = gender_df.iloc[:, 7:].applymap(as_currency)
gender_df.iloc[:, 6:]


,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase Per Person
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,"$1,967.64",$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [18]:
import numpy as np
bins = [0, 9, 14, 19, 24, 29, 34, 39, np.inf]
names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

purchase_data["Age Range"] = pd.cut(purchase_data['Age'], bins, labels=names)

#remove duplicates
no_dup = purchase_data.drop_duplicates(subset = ["SN", "Age"])

age_count = no_dup["Age Range"].value_counts().sort_index()

age_df = pd.DataFrame(age_count)
age_df.columns = ["Count"]

#use total gender from before
age_df["Percentage of Players"] = age_df["Count"] / total_gender
age_df["Percentage of Players"] = pd.Series(["{0:.2f}%".format(val * 100) for val in age_df['Percentage of Players']], index = age_df.index)

age_df


,Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [24]:
age_range_group = purchase_data.groupby("Age Range")
age_range_df = age_range_group.count()
age_range_df["Purchase Count"] = age_range_df["Purchase ID"]
age_range_df["Total Purchase Value"] = age_range_group.sum()["Price"]
age_range_df["Average Purchase Price"] = age_range_df["Total Purchase Value"] / age_range_df["Purchase Count"] 
age_range_df["Avg Total Purchase per Person"] = age_range_df["Total Purchase Value"] / age_df["Count"] #age_df from previous cell

#change format (as_currency function defined earlier) and display
age_range_df.iloc[:, 8:] = age_range_df.iloc[:, 8:].applymap(as_currency)
age_range_df.iloc[:, 7:]

,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Age Range,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,"$1,114.06",$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [36]:
spender_df = sn_gender_group.count()
spender_df["Purchase Count"] = spender_df["Purchase ID"]
spender_df["Total Purchase Value"] = sn_gender_group.sum()["Price"]
spender_df["Avg Purchase Price"] = spender_df["Total Purchase Value"] / spender_df["Purchase Count"]
spender_df = spender_df.sort_values("Total Purchase Value", ascending = False).head(5)

#change format (as_currency function defined earlier) and display
spender_df.iloc[:, 6:] = spender_df.iloc[:, 6:].applymap(as_currency)
spender_df.iloc[:, 5:]

,,Purchase Count,Total Purchase Value,Avg Purchase Price
SN,Gender,,,
Lisosia93,Male,5,$18.96,$3.79
Idastidru52,Male,4,$15.45,$3.86
Chamjask73,Female,3,$13.83,$4.61
Iral74,Male,4,$13.62,$3.40
Iskadarya95,Male,3,$13.10,$4.37


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [43]:
item_group = purchase_data.groupby(["Item ID", "Item Name"])
item_df = item_group.count()
item_df["Purchase Count"] = item_df["Purchase ID"]
item_df["Total Purchase Value"] = item_group.sum()["Price"]
item_df["Avg Item Price"] = item_df["Total Purchase Value"] / item_df["Purchase Count"]
item_popular_df = item_df.sort_values("Purchase Count", ascending = False).head(5)

#change format (as_currency function defined earlier) and display
item_popular_df.iloc[:, 7:] = item_popular_df.iloc[:, 6:].applymap(as_currency)
item_popular_df.iloc[:, 6:]

,,Purchase Count,Total Purchase Value,Avg Item Price
Item ID,Item Name,,,
92,Final Critic,13,$59.99,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
132,Persuasion,9,$28.99,$3.22
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [44]:
item_profitable_df = item_df.sort_values("Total Purchase Value", ascending = False).head(5)

#change format (as_currency function defined earlier) and display
item_profitable_df.iloc[:, 7:] = item_profitable_df.iloc[:, 6:].applymap(as_currency)
item_profitable_df.iloc[:, 6:]

,,Purchase Count,Total Purchase Value,Avg Item Price
Item ID,Item Name,,,
92,Final Critic,13,$59.99,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
103,Singed Scalpel,8,$34.80,$4.35
